<img src=https://www.cct.ie/wp-content/uploads/CCT_Logo_New_Aug_17-2.jpg alt="drawing" style="width:600px;"/>

<div style="text-align: center">

## CCT College Dublin Continuous Assessment

---

**Programme Title/Year:**  	BSc Computing in IT Y4

**Module Title:**	Artificial Intelligence

**Lecturer Name:**	David McQuaid

**Assessment Title:**	AI_Lv8_CA2_v8

**Assessment Type:**	INDIVIDUAL

**Assessment Weighting:**	40% 

**Student Name:**	Leisly Pino

**Student Numbers:**	2020303

**Assessment Due Date:**	17th November 2023

**Date of Submission:**	17th November 2023

---
**Declaration** 
    
<div style="text-align: justify">
By submitting this assessment, I confirm that I have read the CCT policy on Academic Misconduct and understand the implications of submitting work that is not my own or does not appropriately reference material taken from a third party or other source. I declare it to be my own work and that all material from third parties has been appropriately referenced. I further confirm that this work has not previously been submitted for assessment by myself or someone else in CCT College Dublin or any other higher education institution.


----

1. A puzzle has multiple ways of reaching the end solution. Fig. 1 shows a graph that represents all possible routes to the solution. The starting point of the game is represented by A, the solution is represented by S. The other points in the graph are possible intermediary stages. 

<img src=A.jpg/>

<div style="text-align: center">
    Fig. 1 

(a)	The graph in Fig. 1 is a visualisation of the problem. 

(i)	Identify the differences between a graph and a tree. [0-5] 

(ii)	Explain in detail how the graph is an abstraction of the problem. [0-5]

(iii)	Identify the advantages of using a visualisation such as the one shown in Fig. 1. [0-5]

(b)	Demonstrate how Dijkstra’s algorithm would find the shortest path to the solution in Fig.1 through diagrams and written explanation of each stage. [0-25]


In [1]:
import heapq

In [2]:
def calculate_distances(graph, starting_vertex, target_vertex):
    distances = {vertex: float('infinity') for vertex in graph}
    distances[starting_vertex] = 0

    pq = [(0, starting_vertex)]
    while len(pq) > 0:
        current_distance, current_vertex = heapq.heappop(pq)

        # Nodes can get added to the priority queue multiple times. We only
        # process a vertex the first time we remove it from the priority queue.
        if current_distance > distances[current_vertex]:
            continue

        for neighbor, weight in graph[current_vertex].items():
            distance = current_distance + weight

            # Only consider this new path if it's better than any path we've
            # already found.
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(pq, (distance, neighbor))
    path = []
    current_vertex = target_vertex
    while current_vertex != starting_vertex:
        path.insert(0, current_vertex)
        for neighbor, weight in graph[current_vertex].items():
            if distances[current_vertex] == distances[neighbor] + weight:
                current_vertex = neighbor
                break
    path.insert(0, starting_vertex)

    return distances, path

In [3]:
graph1 = {
    'A': {'B': 1, 'C': 5, 'H': 2},
    'B': {'A': 1, 'C': 2, 'D': 4},
    'C': {'A': 5, 'B': 2, 'G': 1},
    'D': {'B': 4, 'E': 4, 'F': 7, 'L': 7},
    'E': {'D': 4, 'F': 3, 'W': 6},
    'F': {'D': 7, 'E': 3, 'G':2},
    'G': {'C': 1, 'F': 2, 'L': 3},
    'H': {'A': 2, 'J': 9, 'K': 5},
    'J': {'H': 9, 'K': 3, 'N': 6},
    'K': {'H': 5, 'J': 3, 'L': 5},
    'L': {'D': 7, 'G': 3, 'K':5, 'M': 4, 'N': 3, 'V': 10, 'W': 8},
    'M': {'L': 4, 'P': 2, 'Q': 10},
    'N': {'J': 6, 'L': 3, 'P': 4, 'S': 7},
    'P': {'M': 2, 'N': 4, 'R': 5},
    'Q': {'M': 10, 'S': 8, 'W': 4},
    'R': {'P': 5, 'S': 4, 'T': 3},
    'S': {'N': 7, 'Q': 8, 'R': 4, 'T': 4, 'U': 2, 'V': 6},
    'T': {'R': 3, 'S': 4, 'U': 1},
    'U': {'S': 2, 'T': 1, 'V': 3},
    'V': {'L': 10, 'S': 6, 'U': 3, 'W': 5},
    'W': {'E': 6, 'L': 8, 'Q': 4, 'V': 5},
}

In [11]:
distances, path = calculate_distances(graph1, 'A', 'S')

print(f"Shortest path: {path}")
print(f"Sum of the shortest path: {distances['S']}")

Shortest path: ['A', 'B', 'C', 'G', 'L', 'N', 'S']
Sum of the shortest path: 17


2. The creator of the puzzle has been told that the A* algorithm is more efficient at finding the shortest path because it uses heuristics. Compare the performance of Dijkstra’s algorithm and the A* search algorithm, referring to heuristics, to find the shortest path to the problem by implementing both algorithms programmatically and comparing the solutions generated in Mark-down. Refer to the complexity of the algorithms and compare the actual time it takes for the solutions to be processed.

In [1]:
from collections import deque

class Graph:

    def __init__(self, adjacency_list):
        self.adjacency_list = adjacency_list

    def get_neighbors(self, v):
        return self.adjacency_list[v]

    # heuristic function
    def h(self, n):
        H = {
            'A': 0,
            'B': 1,
            'C': 2,
            'D': 4,
            'E': 6,
            'F': 5,
            'G': 3,
            'H': 1,
            'J': 5,
            'K': 4,
            'L': 8,
            'M': 10,
            'N': 7,
            'P': 12,
            'Q': 14,
            'R': 16,
            'S': 20,
            'T': 26,
            'U': 24,
            'V': 18,
            'W': 16
        }

        return H[n]

    def a_star_algorithm(self, start_node, stop_node):
        # open_list is a list of nodes which have been visited, but who's neighbors
        # haven't all been inspected, starts off with the start node
        # closed_list is a list of nodes which have been visited
        # and who's neighbors have been inspected
        open_list = set([start_node])
        closed_list = set([])

        # g contains current distances from start_node to all other nodes
        # the default value (if it's not found in the map) is +infinity
        g = {}

        g[start_node] = 0

        # parents contains an adjacency map of all nodes
        parents = {}
        parents[start_node] = start_node

        while len(open_list) > 0:
            n = None

            # find a node with the lowest value of f() - evaluation function
            for v in open_list:
                if n == None or g[v] + self.h(v) < g[n] + self.h(n):
                    n = v;

            if n == None:
                print('Path does not exist!')
                return None

            # if the current node is the stop_node
            # then we begin reconstructin the path from it to the start_node
            if n == stop_node:
                reconst_path = []

                while parents[n] != n:
                    reconst_path.append(n)
                    n = parents[n]

                reconst_path.append(start_node)

                reconst_path.reverse()

                print('Path found: {}'.format(reconst_path))
                return reconst_path

            # for all neighbors of the current node do
            for (m, weight) in self.get_neighbors(n):
                # if the current node isn't in both open_list and closed_list
                # add it to open_list and note n as it's parent
                if m not in open_list and m not in closed_list:
                    open_list.add(m)
                    parents[m] = n
                    g[m] = g[n] + weight

                # otherwise, check if it's quicker to first visit n, then m
                # and if it is, update parent data and g data
                # and if the node was in the closed_list, move it to open_list
                else:
                    if g[m] > g[n] + weight:
                        g[m] = g[n] + weight
                        parents[m] = n

                        if m in closed_list:
                            closed_list.remove(m)
                            open_list.add(m)

            # remove n from the open_list, and add it to closed_list
            # because all of his neighbors were inspected
            open_list.remove(n)
            closed_list.add(n)

        print('Path does not exist!')
        return None

In [4]:
adjacency_list = {
    'A': [('B', 1), ('C', 5), ('H', 2)],
    'B': [('A', 1), ('C', 2), ('D', 4)],
    'C': [('A', 5), ('B', 2), ('G', 1)],
    'D': [('B', 4), ('E', 4), ('F', 7), ('L', 7)],
    'E': [('D', 4), ('F', 3), ('W', 6)],
    'F': [('D', 7), ('E', 3), ('G', 2)],
    'G': [('C', 1), ('F', 2), ('L', 3)],
    'H': [('A', 2), ('J', 9), ('K', 5)],
    'J': [('H', 9), ('K', 3), ('N', 6)],
    'K': [('H', 5), ('J', 3), ('L', 5)],
    'L': [('D', 7), ('G', 3), ('K', 5), ('M', 4), ('N', 3), ('V', 10), ('W', 8)],
    'M': [('L', 4), ('P', 2), ('Q', 10)],
    'N': [('J', 6), ('L', 3), ('P', 4), ('S', 7)],
    'P': [('M', 2), ('N', 4), ('R', 5)],
    'Q': [('M', 10), ('S', 8), ('W', 4)],
    'R': [('P', 5), ('S', 4), ('T', 3)],
    'S': [('N', 7), ('Q', 8), ('R', 4), ('T', 4), ('U', 2), ('V', 6)],
    'T': [('R', 3), ('S', 4), ('U', 1)],
    'U': [('S', 2), ('T', 1), ('V', 3)],
    'V': [('L', 10), ('S', 6), ('U', 3), ('W', 5)],
    'W': [('E', 6), ('L', 8), ('Q', 4), ('V', 5)],
}

graph1 = Graph(adjacency_list)
graph1.a_star_algorithm('A', 'S')

Path found: ['A', 'B', 'C', 'G', 'L', 'N', 'S']


['A', 'B', 'C', 'G', 'L', 'N', 'S']

In [5]:
import heapq
from collections import deque

def calculate_distances(graph, starting_vertex, target_vertex):
    # Dijkstra's Algorithm
    distances_dijkstra = {vertex: float('infinity') for vertex in graph}
    distances_dijkstra[starting_vertex] = 0
    pq = [(0, starting_vertex)]
    
    while pq:
        current_distance, current_vertex = heapq.heappop(pq)

        if current_distance > distances_dijkstra[current_vertex]:
            continue

        for neighbor, weight in graph[current_vertex].items():
            distance = current_distance + weight

            if distance < distances_dijkstra[neighbor]:
                distances_dijkstra[neighbor] = distance
                heapq.heappush(pq, (distance, neighbor))
    
    # A* Search Algorithm
    open_list = deque([starting_vertex])
    closed_list = set([])
    g = {starting_vertex: 0}
    parents = {starting_vertex: starting_vertex}

    while open_list:
        n = open_list.popleft()

        if n == target_vertex:
            break

        for m, weight in graph.get_neighbors(n):
            if m not in open_list and m not in closed_list:
                open_list.append(m)
                parents[m] = n
                g[m] = g[n] + weight
            else:
                if g[m] > g[n] + weight:
                    g[m] = g[n] + weight
                    parents[m] = n

                    if m in closed_list:
                        closed_list.remove(m)
                        open_list.append(m)

        closed_list.add(n)

    # Reconstruct paths
    path_dijkstra = reconstruct_path(parents, starting_vertex, target_vertex)
    path_a_star = graph.a_star_algorithm(starting_vertex, target_vertex)

    return distances_dijkstra[target_vertex], path_dijkstra, path_a_star

def reconstruct_path(parents, start, stop):
    path = []
    current_vertex = stop

    while current_vertex != start:
        path.insert(0, current_vertex)
        current_vertex = parents[current_vertex]

    path.insert(0, start)
    return path

graph1 = {
    'A': {'B': 1, 'C': 5, 'H': 2},
    'B': {'A': 1, 'C': 2, 'D': 4},
    'C': {'A': 5, 'B': 2, 'G': 1},
    'D': {'B': 4, 'E': 4, 'F': 7, 'L': 7},
    'E': {'D': 4, 'F': 3, 'W': 6},
    'F': {'D': 7, 'E': 3, 'G':2},
    'G': {'C': 1, 'F': 2, 'L': 3},
    'H': {'A': 2, 'J': 9, 'K': 5},
    'J': {'H': 9, 'K': 3, 'N': 6},
    'K': {'H': 5, 'J': 3, 'L': 5},
    'L': {'D': 7, 'G': 3, 'K':5, 'M': 4, 'N': 3, 'V': 10, 'W': 8},
    'M': {'L': 4, 'P': 2, 'Q': 10},
    'N': {'J': 6, 'L': 3, 'P': 4, 'S': 7},
    'P': {'M': 2, 'N': 4, 'R': 5},
    'Q': {'M': 10, 'S': 8, 'W': 4},
    'R': {'P': 5, 'S': 4, 'T': 3},
    'S': {'N': 7, 'Q': 8, 'R': 4, 'T': 4, 'U': 2, 'V': 6},
    'T': {'R': 3, 'S': 4, 'U': 1},
    'U': {'S': 2, 'T': 1, 'V': 3},
    'V': {'L': 10, 'S': 6, 'U': 3, 'W': 5},
    'W': {'E': 6, 'L': 8, 'Q': 4, 'V': 5},
}

result_dijkstra = calculate_distances(graph1, 'A', 'S')
print(f"Dijkstra's Algorithm:")
print(f"Shortest path: {result_dijkstra[1]}")
print(f"Sum of the shortest path: {result_dijkstra[0]}")

result_a_star = calculate_distances(graph1, 'A', 'S')
print(f"\nA* Search Algorithm:")
print(f"Shortest path: {result_a_star[2]}")
print(f"Sum of the shortest path: {result_a_star[0]}")


AttributeError: 'dict' object has no attribute 'get_neighbors'

In [27]:
import heapq
from collections import deque
import time

class Graph:

    def __init__(self, adjacency_list):
        self.adjacency_list = adjacency_list

    def get_neighbors(self, v):
        return self.adjacency_list[v]

    def h(self, n):
        H = {
            'A': 0, 'B': 1, 'C': 2, 'D': 4, 'E': 6,
            'F': 5, 'G': 3, 'H': 1, 'J': 5, 'K': 4,
            'L': 8, 'M': 10, 'N': 7, 'P': 12, 'Q': 14,
            'R': 16, 'S': 20, 'T': 26, 'U': 24, 'V': 18,
            'W': 16
        }
        return H[n]

    def a_star_algorithm(self, start_node, stop_node):
        open_list = set([start_node])
        closed_list = set([])

        # g contains current distances from start_node to all other nodes
        # the default value (if it's not found in the map) is +infinity
        g = {}

        g[start_node] = 0

        # parents contains an adjacency map of all nodes
        parents = {}
        parents[start_node] = start_node

        while len(open_list) > 0:
            n = None

            # find a node with the lowest value of f() - evaluation function
            for v in open_list:
                if n == None or g[v] + self.h(v) < g[n] + self.h(n):
                    n = v;

            if n == None:
                print('Path does not exist!')
                return None

            # if the current node is the stop_node
            # then we begin reconstructin the path from it to the start_node
            if n == stop_node:
                reconst_path = []

                while parents[n] != n:
                    reconst_path.append(n)
                    n = parents[n]

                reconst_path.append(start_node)

                reconst_path.reverse()

                print('Path found: {}'.format(reconst_path))
                return reconst_path

            # for all neighbors of the current node do
            for (m, weight) in self.get_neighbors(n):
                # if the current node isn't in both open_list and closed_list
                # add it to open_list and note n as it's parent
                if m not in open_list and m not in closed_list:
                    open_list.add(m)
                    parents[m] = n
                    g[m] = g[n] + weight

                # otherwise, check if it's quicker to first visit n, then m
                # and if it is, update parent data and g data
                # and if the node was in the closed_list, move it to open_list
                else:
                    if g[m] > g[n] + weight:
                        g[m] = g[n] + weight
                        parents[m] = n

                        if m in closed_list:
                            closed_list.remove(m)
                            open_list.add(m)

            # remove n from the open_list, and add it to closed_list
            # because all of his neighbors were inspected
            open_list.remove(n)
            closed_list.add(n)

        print('Path does not exist!')
        return None

def dijkstra(self, starting_vertex, target_vertex):
    distances = {vertex: float('infinity') for vertex in self.adjacency_list.keys()}
    distances[starting_vertex] = 0
    pq = [(0, starting_vertex)]

    while pq:
        current_distance, current_vertex = heapq.heappop(pq)

        if current_distance > distances[current_vertex]:
            continue

        for neighbor, weight in self.get_neighbors(current_vertex):
            distance = current_distance + weight

            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(pq, (distance, neighbor))

    path = []
    current_vertex = target_vertex
    while current_vertex != starting_vertex:
        path.insert(0, current_vertex)
        for neighbor, weight in self.get_neighbors(current_vertex):
            if distances[current_vertex] == distances[neighbor] + weight:
                current_vertex = neighbor
                break
    path.insert(0, starting_vertex)

    return distances, path


def compare_algorithms(graph, starting_vertex, target_vertex):
    # Medir tiempo de ejecución de Dijkstra
    start_time_dijkstra = time.time()
    distances_dijkstra, path_dijkstra = dijkstra(graph, starting_vertex, target_vertex)
    end_time_dijkstra = time.time()
    time_dijkstra = end_time_dijkstra - start_time_dijkstra

    # Medir tiempo de ejecución de A*
    start_time_a_star = time.time()
    graph_a_star = Graph(graph.adjacency_list)
    path_a_star = graph_a_star.a_star_algorithm(starting_vertex, target_vertex)
    end_time_a_star = time.time()
    time_a_star = end_time_a_star - start_time_a_star

    # Mostrar resultados en formato Markdown
    print("### Resultados de Dijkstra's Algorithm:")
    print(f"**Shortest path:** {path_dijkstra}")
    print(f"**Sum of the shortest path:** {distances_dijkstra[target_vertex]}")
    print(f"**Tiempo de ejecución:** {time_dijkstra:.20f} segundos\n")

    print("### Resultados de A* Algorithm:")
    print(f"**Shortest path:** {path_a_star}")
    print(f"**Sum of the shortest path:** {sum(graph.h(node) for node in path_a_star)}")
    print(f"**Tiempo de ejecución:** {time_a_star:.20f} segundos")

# Define el grafo
adjacency_list = {
    'A': [('B', 1), ('C', 5), ('H', 2)],
    'B': [('A', 1), ('C', 2), ('D', 4)],
    'C': [('A', 5), ('B', 2), ('G', 1)],
    'D': [('B', 4), ('E', 4), ('F', 7), ('L', 7)],
    'E': [('D', 4), ('F', 3), ('W', 6)],
    'F': [('D', 7), ('E', 3), ('G', 2)],
    'G': [('C', 1), ('F', 2), ('L', 3)],
    'H': [('A', 2), ('J', 9), ('K', 5)],
    'J': [('H', 9), ('K', 3), ('N', 6)],
    'K': [('H', 5), ('J', 3), ('L', 5)],
    'L': [('D', 7), ('G', 3), ('K', 5), ('M', 4), ('N', 3), ('V', 10), ('W', 8)],
    'M': [('L', 4), ('P', 2), ('Q', 10)],
    'N': [('J', 6), ('L', 3), ('P', 4), ('S', 7)],
    'P': [('M', 2), ('N', 4), ('R', 5)],
    'Q': [('M', 10), ('S', 8), ('W', 4)],
    'R': [('P', 5), ('S', 4), ('T', 3)],
    'S': [('N', 7), ('Q', 8), ('R', 4), ('T', 4), ('U', 2), ('V', 6)],
    'T': [('R', 3), ('S', 4), ('U', 1)],
    'U': [('S', 2), ('T', 1), ('V', 3)],
    'V': [('L', 10), ('S', 6), ('U', 3), ('W', 5)],
    'W': [('E', 6), ('L', 8), ('Q', 4), ('V', 5)],
}

# Crea una instancia del grafo
graph = Graph(adjacency_list)

# Ejecutar la comparación de algoritmos
compare_algorithms(graph, 'A', 'S')


Path found: ['A', 'B', 'C', 'G', 'L', 'N', 'S']
### Resultados de Dijkstra's Algorithm:
**Shortest path:** ['A', 'B', 'C', 'G', 'L', 'N', 'S']
**Sum of the shortest path:** 17
**Tiempo de ejecución:** 0.00000000000000000000 segundos

### Resultados de A* Algorithm:
**Shortest path:** ['A', 'B', 'C', 'G', 'L', 'N', 'S']
**Sum of the shortest path:** 41
**Tiempo de ejecución:** 0.00000000000000000000 segundos


In [ ]:
import heapq

def calculate_distances_with_heuristic(graph, starting_vertex, target_vertex, heuristic):
    distances = {vertex: float('infinity') for vertex in graph}
    distances[starting_vertex] = 0

    pq = [(0, starting_vertex)]
    while len(pq) > 0:
        current_distance, current_vertex = heapq.heappop(pq)

        if current_distance > distances[current_vertex]:
            continue

        for neighbor, weight in graph[current_vertex].items():
            distance = current_distance + weight

            # Incorporar la heurística
            distance_with_heuristic = distance + heuristic[neighbor]

            if distance_with_heuristic < distances[neighbor]:
                distances[neighbor] = distance_with_heuristic
                heapq.heappush(pq, (distance_with_heuristic, neighbor))

    path = []
    current_vertex = target_vertex
    while current_vertex != starting_vertex:
        path.insert(0, current_vertex)
        for neighbor, weight in graph[current_vertex].items():
            if distances[current_vertex] == distances[neighbor] - heuristic[neighbor] + weight:
                current_vertex = neighbor
                break
    path.insert(0, starting_vertex)

    return distances, path

# Definición del grafo
graph1 = {
    'A': {'B': 1, 'C': 5, 'H': 2},
    'B': {'A': 1, 'C': 2, 'D': 4},
    'C': {'A': 5, 'B': 2, 'G': 1},
    'D': {'B': 4, 'E': 4, 'F': 7, 'L': 7},
    'E': {'D': 4, 'F': 3, 'W': 6},
    'F': {'D': 7, 'E': 3, 'G': 2},
    'G': {'C': 1, 'F': 2, 'L': 3},
    'H': {'A': 2, 'J': 9, 'K': 5},
    'J': {'H': 9, 'K': 3, 'N': 6},
    'K': {'H': 5, 'J': 3, 'L': 5},
    'L': {'D': 7, 'G': 3, 'K': 5, 'M': 4, 'N': 3, 'V': 10, 'W': 8},
    'M': {'L': 4, 'P': 2, 'Q': 10},
    'N': {'J': 6, 'L': 3, 'P': 4, 'S': 7},
    'P': {'M': 2, 'N': 4, 'R': 5},
    'Q': {'M': 10, 'S': 8, 'W': 4},
    'R': {'P': 5, 'S': 4, 'T': 3},
    'S': {'N': 7, 'Q': 8, 'R': 4, 'T': 4, 'U': 2, 'V': 6},
    'T': {'R': 3, 'S': 4, 'U': 1},
    'U': {'S': 2, 'T': 1, 'V': 3},
    'V': {'L': 10, 'S': 6, 'U': 3, 'W': 5},
    'W': {'E': 6, 'L': 8, 'Q': 4, 'V': 5},
}

# Heurística para A*
heuristic_a_star = {
    'A': 20, 'B': 18, 'C': 14, 'D': 13, 'E': 12,
    'F': 12, 'G': 13, 'H': 24, 'J': 16, 'K': 15,
    'L': 10, 'M': 8, 'N': 10, 'P': 6, 'Q': 8,
    'R': 10, 'S': 0, 'T': 4, 'U': 6, 'V': 5, 'W': 2
}

# Calcular distancias con heurística combinada con Dijkstra
distances_combined, path_combined = calculate_distances_with_heuristic(graph1, 'A', 'S', heuristic_a_star)

print(f"Combined (Dijkstra + Heuristic) Shortest path: {path_combined}")
print(f"Sum of Combined shortest path: {distances_combined['S']}")


<table>
        <tr>
            <th style="text-align: center"><img src=2.jpg width= 400px /></th>
            <th style="text-align: center"><img src=3.jpg width= 400px /></th>
            <th style="text-align: center"><img src=4.jpg width= 400px /></th>
        </tr>
</table>